In [ ]:
import fitz  # PyMuPDF
import os


def extract_images_from_pdf(pdf_path, output_dir, start_page_num=1):
    """
    PDF에서 이미지를 추출하고, 지정된 시작 페이지 번호로 파일 저장
    :param pdf_path: PDF 파일 경로
    :param output_dir: 이미지를 저장할 디렉토리
    :param start_page_num: 저장할 파일의 시작 페이지 번호 (기본값: 1)
    :return: 추출된 이미지 경로 리스트
    """
    # PDF 열기
    pdf_document = fitz.open(pdf_path)
    os.makedirs(output_dir, exist_ok=True)
    image_paths = []

    current_page_num = start_page_num  # 저장할 파일의 페이지 번호

    for page_index in range(len(pdf_document)):
        page = pdf_document[page_index]
        images = page.get_images(full=True)  # 페이지에서 이미지 정보 가져오기

        if not images:
            print(f"Skipping page {page_index + 1}: No images found.")
            continue  # 이미지가 없는 페이지는 건너뛰기

        for img_index, img in enumerate(images):
            xref = img[0]  # 이미지 xref 값
            base_image = pdf_document.extract_image(xref)  # 이미지 추출
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]  # 이미지 확장자 (png, jpeg 등)

            # 저장할 파일 이름 지정 (시작 페이지 번호부터 증가)
            image_path = os.path.join(
                output_dir, f"{current_page_num}_{img_index + 1}.{image_ext}"
            )

            # 이미지 파일 저장
            with open(image_path, "wb") as image_file:
                image_file.write(image_bytes)
            image_paths.append(image_path)
            print(f"Extracted image saved as: {image_path}")

        current_page_num += 1  # 다음 저장 페이지 번호 증가

    pdf_document.close()
    return image_paths


# PDF 경로와 이미지 저장 디렉토리
pdf_path = "./data/pdf_data/education_culture_center.pdf"
output_dir = "./data/images/education_culture_center"

# 저장 시작 페이지 번호 지정
start_page_num = 72  # 파일 이름을 5번 페이지부터 시작

# 이미지 추출
images = extract_images_from_pdf(pdf_path, output_dir, start_page_num)
print(f"Extracted images: {images}")

In [ ]:
import os
import re


def rename_images_in_folder(folder_path):
    # 폴더 내 모든 파일 가져오기
    files = sorted(os.listdir(folder_path))

    # 정규식을 사용하여 파일 이름에서 페이지 번호와 이미지 번호 추출
    pattern = r"(\d+)_(\d+)\.(\w+)"  # ex: 1_2.jpg, 2_3.png

    # 페이지 번호별로 파일을 그룹화
    page_groups = {}
    for file_name in files:
        match = re.match(pattern, file_name)
        if match:
            page_num, img_num, extension = match.groups()
            if page_num not in page_groups:
                page_groups[page_num] = []
            page_groups[page_num].append((file_name, int(img_num), extension))

    # 각 페이지 그룹의 파일 번호를 재정렬
    for page_num, images in page_groups.items():
        # 이미지 번호 기준으로 정렬
        images.sort(key=lambda x: x[1])

        # 새 번호로 재배치
        for new_img_num, (file_name, _, extension) in enumerate(images, start=1):
            # 기존 파일 경로와 새 파일 경로 생성
            old_path = os.path.join(folder_path, file_name)
            new_file_name = f"{page_num}_{new_img_num}.{extension}"
            new_path = os.path.join(folder_path, new_file_name)

            # 파일 이름 변경
            os.rename(old_path, new_path)
            print(f"Renamed: {file_name} -> {new_file_name}")


# 폴더 경로 설정
folder_path = "./data/images/middle_gate"

# 파일 이름 변경 실행
rename_images_in_folder(folder_path)

In [ ]:
import boto3
import os
import re

# S3 클라이언트 생성
s3 = boto3.client("s3")
BUCKET_NAME = "{버킷명}"
BASE_FOLDER = "rooms/"  # S3 내 기본 폴더


def upload_images_and_create_empty_folders(folder_paths, room_ids):
    """
    여러 로컬 폴더에서 S3로 이미지를 업로드하고, 이미지가 없는 room_id에 빈 폴더 생성
    :param folder_paths: 로컬 폴더 경로 리스트
    :param room_ids: 대상 room_id 리스트
    """
    # 정규식으로 파일 이름에서 room_id와 이미지 번호 추출
    pattern = r"(\d+)_(\d+)\.(\w+)"

    for room_id in room_ids:
        room_folder = f"{BASE_FOLDER}{room_id}/"  # S3의 room_id 폴더 경로
        has_images = False  # 해당 room_id에 이미지가 있는지 여부

        for folder_path in folder_paths:
            if not os.path.exists(folder_path):
                print(f"Folder does not exist: {folder_path}")
                continue

            # 폴더 내 파일 가져오기
            files = os.listdir(folder_path)

            for file_name in files:
                match = re.match(pattern, file_name)
                if match:
                    file_room_id, img_num, extension = match.groups()

                    # 현재 room_id에 해당하는 파일만 업로드
                    if file_room_id == str(room_id):
                        local_path = os.path.join(folder_path, file_name)
                        s3_key = f"{room_folder}{file_name}"  # S3에 업로드할 전체 경로

                        # Content-Type 설정
                        content_type_map = {
                            "jpg": "image/jpeg",
                            "jpeg": "image/jpeg",
                            "png": "image/png",
                            "gif": "image/gif",
                        }
                        content_type = content_type_map.get(
                            extension.lower(), "binary/octet-stream"
                        )

                        # S3에 파일 업로드 (Content-Type 설정 추가)
                        s3.upload_file(
                            Filename=local_path,
                            Bucket=BUCKET_NAME,
                            Key=s3_key,
                            ExtraArgs={
                                "ContentType": content_type
                            },  # Content-Type 설정
                        )
                        print(
                            f"Uploaded {file_name} to s3://{BUCKET_NAME}/{s3_key} with Content-Type: {content_type}"
                        )
                        has_images = True  # 이미지가 있음을 표시

        # 이미지가 없는 경우 S3에 빈 폴더 생성
        if not has_images:
            # 빈 폴더를 생성하려면 폴더 경로에 0바이트 객체 업로드
            s3.put_object(Bucket=BUCKET_NAME, Key=f"{room_folder}")
            print(f"Created empty folder for room_id {room_id}")


# 로컬 이미지 폴더 경로 리스트
local_folder_paths = [
    "./data/images/back_gate",
    "./data/images/dormitory",
    "./data/images/education_culture_center",
    "./data/images/farming_mart",
    "./data/images/main_gate",
    "./data/images/middle_gate",
]

# room_id 범위
room_ids = range(1, 81)  # 1부터 80까지

# 이미지 업로드 및 빈 폴더 생성 실행
upload_images_and_create_empty_folders(local_folder_paths, room_ids)

In [ ]:
import boto3
import json

# S3 클라이언트 생성
s3 = boto3.client("s3")
BUCKET_NAME = "campus-meet-bucket"
BASE_FOLDER = "rooms/"  # S3 내 기본 폴더 경로
REGION = "ap-northeast-2"  # S3 버킷이 위치한 AWS 리전


def list_images_in_room(bucket_name, room_id):
    folder_prefix = f"{BASE_FOLDER}{room_id}/"  # room_id에 해당하는 폴더 경로
    images = []

    # S3에서 특정 폴더의 객체 리스트 가져오기
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

    # 객체 리스트에서 파일 경로만 추출
    if "Contents" in response:
        for obj in response["Contents"]:
            if not obj["Key"].endswith("/"):  # 폴더 자체는 제외
                images.append(obj["Key"])  # S3 객체 키 (파일 경로)

    return images


def generate_image_urls(bucket_name, region, keys):
    urls = [f"https://{bucket_name}.s3.{region}.amazonaws.com/{key}" for key in keys]
    return urls


# 예시: room_id가 1인 경우
room_id = "1"
keys = list_images_in_room(BUCKET_NAME, room_id)

# S3 객체 키를 URL로 변환
image_urls = generate_image_urls(BUCKET_NAME, REGION, keys) if keys else []

# JSON 응답 생성
response_data = {"room_id": room_id, "images": image_urls}
response_json = json.dumps(
    response_data, indent=4
)  # JSON 문자열로 변환 (보기 좋게 들여쓰기 포함)

# 클라이언트에게 전송 (예: 출력)
print(response_json)

In [ ]:
import pandas as pd
import requests
import time

# Load the CSV file
file_path = "./data/csv_data/rental_data_with_null.csv"
data = pd.read_csv(file_path)

# Google Maps Geocoding API 설정
API_KEY = ""  # 여기에 Google Maps API 키를 입력하세요
GEOCODING_URL = "https://maps.googleapis.com/maps/api/geocode/json"


# Function to get latitude and longitude based on address using Google Maps API
def get_lat_lon_google(address):
    params = {"address": address, "key": API_KEY}
    try:
        response = requests.get(GEOCODING_URL, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "OK" and len(data["results"]) > 0:
                location = data["results"][0]["geometry"]["location"]
                return location["lat"], location["lng"]
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching coordinates for address '{address}': {e}")
        return None, None


# Add latitude and longitude columns
data["latitude"] = None
data["longitude"] = None

# Fetch latitude and longitude for each address
for index, row in data.iterrows():
    if pd.notna(row["address"]):  # Ensure the address is not NaN
        lat, lon = get_lat_lon_google(row["address"])
        data.at[index, "latitude"] = lat
        data.at[index, "longitude"] = lon
        print(f"Processed {row['address']}: Latitude={lat}, Longitude={lon}")
        time.sleep(0.1)  # Rate limiting to avoid API quota exhaustion

# Save the updated DataFrame to a new CSV file
output_file_path = "./data/csv_data/rental_data_with_lat_lon_google.csv"
data.to_csv(output_file_path, index=False)

print(f"Updated CSV saved to {output_file_path}")